# Homework 3: Discover, Measure, and Mitigate Bias in Bank Marketing

## Background

In this homework, we use a data coming from a bank’s marketing campaign. It consists of several individual level variables like age, gender, credit default, job etc., which can serve as input variables in the prediction model. The outcome varaible that the bank is interested in is whether a person subscribed to the term deposit or not. Hence, the outcome variable is categorical in nature ‐ subscribed or did not subscribe. The objective of training a model is to predict if someone would subscribe to the term deposit oﬀered by the bank or not. Given that the cost and time to contact all possible leads is enormous hence, ﬁnancial institutions like to identify the most promising leads. Promising leads are likely to be identiﬁed as proﬁle of people who are most likely to subscribe to a term deposit. Once identiﬁed, these leads are contacted through direct marketing channels (e.g., phone calls), they are provided with all the details about the term deposit.

But the bank also wants to make sure that the prediction model is not biased against any group. They are cognizant that a prediction model built on prior data set has the potential to display bias against diﬀerent groups which precludes them from appearing in the list of promising leads. Considering that term deposits can help secure ﬁnancial stability in the long term, a biased prediction model can adversely aﬀect some groups. For the purpose of this project, we will consider marital status (married, not married) as the protected variable of interest. We will refer to the married people as the privileged group and examine whether there is diﬀerences in the privileged group versus the unprivileged group.

| Protected Variable|Privileged Group|Unprivileged Group|
| ----------------- | -------------- | ---------------- |
|   Marital status  |     Married    |    Unmarried     |



## Data Description
The dataset consists of $5000$ rows and $12$ kinds of variables. Run the code below to look at the first $100$ instances of the data.

In [ ]:
import pandas as pd
bank_data = pd.read_csv('bank.csv')
bank_data.head(n=100)

In this data, we refer to the first 11 variables as **input variables** that are the input of the predicton model. The **outcome variable** of $subscribed$ denotes if the client has subscribed to a term deposit. For ease of explanation, we will refer to the two classes of the outcome variable as yes versus no indicating whether a person subscribed (yes) or did not subscribe (no). All variables are:

* $age$: How old this client is. 
* $job$: Type of job. 
* $marital$: Marital status.
* $education$: Highest education.
* $default$: Has credit in default.
* $housing$: Has housing loan?
* $loan$: Has personal loan? 
* $contact$: Contact communication type.
* $month$: Last contact month of year.
* $day\_of\_week$: Last contact day of the week.
* $duration$: Last contact duration, in seconds.
* $subscribed$: Has the client subscribed a term deposit？ 

## Machine Learning Pipeline


![image](../Images/MLWorkflow.png)
We will use the bank data to review basics of how a predicted model is created in a supervised machine learning process.

First, we split the initial dataset into training and test datasets with a random partitioning. Second, a machine learning algorithm is trained on this training dataset to produce a machine learning model. This generated model can be used to make a prediction when given a new instance. Next, the test dataset is used to assess the metrics the model, such as accuracy and fairness.

Bias can enter the system in any of the three steps above. The training data set may be biased in that its outcome variable may be biased towards particular group of instances. The algorithm that creates the model may be biased in that it may generate models that are weighted towards particular variables in the input. The test data set may be biased in that it has expectations on correct answers that may be biased. Corresponding to the three kinds of the source of bias, researchers proposed three types of techniques to mitigate the bias: Pre-processing, In-processing, and Post-processing.

## Steps to Discover, Measure, and Mitigate Bias

We are now ready to build a prediction model on the bank dataset, then discover, measure, and mitigate bias against the unmarried group in this model.

First, we set the protected attribute as $marital$, with "Married" and "Unmarried" being the values for the privileged and unprivileged groups, respectively. Second, we split the Bank Marketing data into training data and test data with a specific split ratio. For example, a split ratio $0.7$ means that a random $70$ of the data goes to the training data, and the remaining becomes the test data. Third, we will build a prediction model on the training data using the Logistic Regression algorithm. Fourth, we get the prediction with the model on test data and check the accuracy and fairness of the prediction; we refer to them as the **baseline** as we don't apply any debiasing techniques so far. Fifth, we will apply Pre-processing, In-processing, and Post-processing techniques to mitigate biases in prediction, then check the accuracy and fairness of the debiased prediction again and compare them with the baseline. Finally, we will combine different debiasing techniques.

Here are the steps involved:

**1. Import libraries.**

**2. Specify protected variable, privileged group, and unprivileged group.**

**3. Split the Bank Marketing data into training data and test data.**

**4. Build a prediction model.**

    4.1 Train a Logistic Regression model using the training data.
    4.2 Make prediction of the test data with the trained model.
    4.3 Check fairness metrics and accuracy of the predition. (Baseline)
    
**5. Apply debiasing techniques to mitigate biases in prediction.**

    5.1 Pre-processing techniques
    5.2 In-processing techniques
    5.3 Post-processing techniques
**6. Flexibly combine different debiasing techniques to mitigate biases in prediction.**

**7. Answer questions**


### Step 1. Import libraries

In [ ]:
from aif360.datasets import BankDataset
from aif360.metrics import BinaryLabelDatasetMetric
from sklearn.linear_model import LogisticRegression
from aif360.metrics import ClassificationMetric
from aif360.algorithms.preprocessing import Reweighing
from aif360.algorithms.preprocessing import LFR
from aif360.algorithms.inprocessing import PrejudiceRemover
from aif360.algorithms.postprocessing import RejectOptionClassification
from IPython.display import Markdown, display
print('If there are warnings, please ignore them.')

### Step 2. Load the bank data, specify protected variable, privileged group, and unprivileged group

In [ ]:
protected_attribute_maps = [{1.0: 'married', 0.0: 'unmarried'}]
dataset_orig = BankDataset(    # load the bank data
            protected_attribute_names=['marital'],    # set the protected attribute as 'marital'
            privileged_classes=[['married']],    # set 'married' as privileged group,
                                                 # 'unmarried' will be unprivileged group automatically
            features_to_drop=['campaign', 'pdays', 'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed'],
            categorical_features=['job', 'education', 'default',
                    'housing', 'loan', 'contact', 'month', 'day_of_week'],
            metadata={'protected_attribute_maps': protected_attribute_maps}
        )
privileged_groups = [{'marital': 1}]
unprivileged_groups = [{'marital': 0}]

display(Markdown("#### The total number of instances in the bank data"))
print(dataset_orig.features.shape[0])
display(Markdown("#### Protected variable name"))
print(dataset_orig.protected_attribute_names)

### Step 3. Split the Bank Marketing data into training data and test data

We set the split ratio as $0.7$ by default, meaning that $70\%$ instances in the dataset become the training data, and the remaining $30\%$ instances become the test data. You can try different split ratio by changing the value of `split_ratio`, but we recommand $0.6-0.8$.

In [ ]:
split_ratio = 0.7
dataset_orig_train, dataset_orig_test = dataset_orig.split([split_ratio], shuffle=None)
display(Markdown("#### Split results"))
print("Split ratio = %f" % split_ratio)
print("The total number of instances in the bank data = %d" % dataset_orig.features.shape[0])
print("The number of instances in the training data = %d" % dataset_orig_train.features.shape[0])
print("The number of instances in the test data = %d" % dataset_orig_test.features.shape[0])

### Step 4. Build a prediction model

#### 4.1 & 4.2  Train a Logistic Regression model using the training data;  Make prediction of the test data with the trained model.

In [ ]:
def Logistic_Regression(training_data, test_data):
    model = LogisticRegression(random_state=0, max_iter = 1000)
    # train the Logistic Regression model using the training data
    model.fit(training_data.features, training_data.labels.ravel(), training_data.instance_weights)
    # get the prediction on the test data using the model
    prediction_label = model.predict(test_data.features)
    prediction = dataset_orig_test.copy()
    prediction.labels = prediction_label
    # return the prediction
    return prediction

# get the baseline prediction of test data on the Logistic Regression model trained by the training data
baseline_prediction = Logistic_Regression(dataset_orig_train, dataset_orig_test)

display(Markdown("#### Prediction result of each instance in the test data"))
print(baseline_prediction.labels)
print()
print('Note: 0: The client subscribed a term deposit; 1: The client doesn\'t subscribed a term deposit')

#### 4.3 Check fairness metrics and accuracy of the predition. (Baseline)
Then, we will check the accuracy of the prediction, and measure the fairness of the prediction using four fairness metrics: **Disparate impact**, **Statistical Parity Difference**, **Equal Opportunity Difference**, and **Average Odds Difference**. Please keep in mind that we will treat values of fairness metrics and accuracy as our baseline, because we haven't applied any debiasing techniques so far.

In [ ]:
# measure the accuracy and the fairness metrics on the prediction
def get_prediction_metrics(prediction):
    metric = ClassificationMetric(
                        dataset_orig_test, prediction,
                        unprivileged_groups=unprivileged_groups,
                        privileged_groups=privileged_groups)
    
    print("Accuracy = %s" % round(metric.accuracy(), 3))
    print("Fairness metrics:")
    print("Disparate Impact = %s" % round(metric.disparate_impact(), 3))
    print("Statistical Parity Difference = %s" % round(metric.statistical_parity_difference(), 3))
    print("Equal Opportunity Difference = %s" % round(metric.equal_opportunity_difference(), 3))
    print("Average Odds Difference = %s" % round(metric.average_odds_difference(), 3))

display(Markdown("#### Baseline of accuracy and the fairness metrics on the prediction"))

# measure the accuracy and fairness of the prediction
get_prediction_metrics(baseline_prediction)

### Step 5. Apply debiasing techniques to mitigate biases in prediction.

In this step, we will try three types of debiasing techniques to mitigate biases: **Pre-processing**, **In-processing**, and **Post-processing**. For each type, we will use 1-2 methods. 

#### 5.1 Pre-processing
Pre‐processing methods attempt to correct the bias by assigning appropriate weights to data points or by transforming them in such a way that discrimination can be reduced. The main idea of pre‐processing methods is to train a model on a “repaired” data set. In this section we will discuss two pre‐processing methods.

**Pre-processing method 1: Reweighing method**

In [ ]:
# Pre-processing: reweighing method
# This method requires the training data as input, then apply reweighing method on it, 
# returned the debiased training data.
def Reweighing_method(training_data):
    RW_model = Reweighing(unprivileged_groups=unprivileged_groups,
            privileged_groups=privileged_groups)
    return RW_model.fit_transform(training_data)

# debiased training data using reweighing method
dataset_RW_train = Reweighing_method(dataset_orig_train)

# Train the Logistic Regression model with the debiased training data, then get the prediction on the test data
prediction = Logistic_Regression(dataset_RW_train, dataset_orig_test)

display(Markdown("#### Accuracy and the fairness metrics on the prediction of test data \n * Training data: debiased training data using reweighing algorithm. \n * Model: Logistic Regression model"))

# measure the accuracy and fairness of the prediction
get_prediction_metrics(prediction)

**Pre-processing method 2: Learning fair representations (LFR)**

In [ ]:
# Pre-processing: Learning fair representations
# This method requires the training data as input, then apply LFR method on it, 
# returned the debiased training data.
def LFR_method(training_data):
    LFR_model = LFR(unprivileged_groups=unprivileged_groups, 
    privileged_groups=privileged_groups,
    verbose=0, seed=10)
    LFR_model = LFR_model.fit(dataset_orig_train)
    return LFR_model.transform(dataset_orig_train)

print('please wait for the results, this process may take 20 senconds ...')

# debiased training data using LFR method
dataset_LFR_train = LFR_method(dataset_orig_train)

# Train the Logistic Regression model with the debiased training data, then get the prediction on the test data
prediction = Logistic_Regression(dataset_LFR_train, dataset_orig_test)

display(Markdown("#### Accuracy and the fairness metrics on the prediction of test data \n * Training data: debiased training data using Learning fair representations (LFR) algorithm. \n * Model: Logistic Regression model"))

# measure the accuracy and fairness of the prediction
get_prediction_metrics(prediction)


#### 5.2 In-processing
In‐processing debiasing occurs during the training process in which a method is attempting to learn the relationship between the input features and the outcome. The goal is to reduce the reliance on learning the relationship between protected attributes and the outcome.
Put it short, In-processing methods just replace the original model with a new model that is fairer.

**In-processing method 1: Prejudice Remover**

In [ ]:
# in-processing: Prejudice remover
# This method take the training data and test data as input
# The training data is used to train the Prejudice remover model, 
# then return the prediction of the test data on the trained model.
def Prejudice_Remover(training_data, test_data):
    model = PrejudiceRemover(eta=0.1)
    model.fit(training_data)
    prediction = model.predict(test_data)
    return prediction

print('please wait for the results, this process may take 20 senconds ...')

# Train Prejudice Remover model using the orginal training data, then return the predition on the original test data.
prediction = Prejudice_Remover(dataset_orig_train, dataset_orig_test)

display(Markdown("#### Accuracy and the fairness metrics on the prediction of test data \n * Training data: original training data. \n * Model: Prejudice Remover model"))

# measure the accuracy and fairness of the prediction
get_prediction_metrics(prediction)

**In-processing method 2: Adversarial Debiasing**

In [ ]:
# in-processing: Adversarial debiasing
from aif360.algorithms.inprocessing.adversarial_debiasing import AdversarialDebiasing
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()

# This method take the training data and test data as input
# The training data is used to train the Adversarial debiasing model, 
# then return the prediction of the test data on the trained model.
def Adversarial_debiasing(training_data, test_data):
    tf.reset_default_graph()
    sess = tf.Session()
    num_epochs = 50
    classifier_num_hidden_units = 200
    model = AdversarialDebiasing(privileged_groups = privileged_groups,
                                unprivileged_groups = unprivileged_groups,
                                scope_name='debiased_classifier',
                                debias=True,
                                sess=sess)
    model.fit(training_data)
    prediction = model.predict(test_data)
    return prediction

# Train Adversarial debiasing model using the orginal training data, then return the predition on the original test data.
prediction = Adversarial_debiasing(dataset_orig_train, dataset_orig_test)

display(Markdown("#### Accuracy and the fairness metrics on the prediction of test data \n * Training data: original training data. \n * Model: Adversarial debiasing"))

# measure the accuracy and fairness of the prediction
get_prediction_metrics(prediction)

#### Post-processing
The post-processing approach mainly focus on modifying the prediction result to make it fairer. In this section, we will use one popular technique, called Reject Option‐based Classiﬁcation (ROC), to ajust the baseline prediction so that it become fairer.

In [ ]:
# Post-processing: Reject Option‐based Classiﬁcation (ROC)

def ROC(prediction):
    model = RejectOptionClassification(privileged_groups = privileged_groups,
                                    unprivileged_groups = unprivileged_groups, num_class_thresh=500)
    model = model.fit(dataset_orig_test, prediction)
    prediction = model.predict(prediction)
    return prediction

print('please wait for the results, this process may take 20 senconds ...')

# get the results after applying Reject Option‐based Classiﬁcation to the prediction baseline
prediction = ROC(baseline_prediction)

display(Markdown("#### Accuracy and the fairness metrics of the prediction after applying Reject Option‐based Classiﬁcation"))

# measure the accuracy and fairness of the prediction
get_prediction_metrics(prediction)

### Step 6. Flexibly combine different debiasing techniques to mitigate biases.
In **step 5**, we show the uses of debiasing techniques individually, including pre-processing, in-processing, and post-processing. In this step, we will show how to combine different debiasing techniques to mitigate baises. In the following code, 
* we apply the **Reweighing** (pre-processing) algorithm on the original training data, 
* and use the debiased training data to train a fairer model **Prejudice Remover** (in-processing), 
* then get the prediction of test data on the trained model, 
* finally, we use **Reject Option‐based Classiﬁcation** (post-processing) to debaised the prediction.
* We evaluate the accuracy and fairness of the final prediction

In [ ]:
### This code shows how to combine three debiasing methdods together: Reweighing, Prejudice Remover, and Reject Option‐based Classiﬁcation

print('please wait for the results, this process may take 30 senconds ...')

# 1. pre-processing: debias the original training data using Reweighing
dataset_RW_train = Reweighing_method(dataset_orig_train)
# 2. in-processing: train Prejudice Remover model using debiased training data, then get the prediction of test data
prediction = Prejudice_Remover(dataset_RW_train, dataset_orig_test)
# 3. post-processing: use Reject Option‐based Classiﬁcation to debaised the prediction
final_prediction = ROC(prediction)

display(Markdown("#### Accuracy and the fairness metrics of the prediction debaised by Reweighing, Prejudice Remover, and Reject Option‐based Classiﬁcation"))
# 4. evaluate the accuracy and fairness of the final prediction
get_prediction_metrics(final_prediction)

### Step 7. Answer questions
Please answer the following questions by editing or runing the code above.

**Q1 (10 points)**: In **step 3**, change the value of the `split_ratio` variable (recommend 0.6-0.8), and run the code in **step 3** and **step 4** to build a Logistic Regression model and make predictions on the test data. Report the value of `split_ratio`, the accuracy, and the four fairness metrics of the predictions (baseline). 

**Q2 (20 points)**: Run the code in **step 5.1**. For each pre-processing method (*Reweighing method* and *Learning fair representations*), report the accuracy and four fairness metrics of the debiased predictions. Compare the results with the baseline generated in **Q1**, How do the accuracy and fairness metrics change？

**Q3 (20 points)**: Run the code in **step 5.2**. For each in-processing method (*Prejudice Remover* and *Adversarial Debiasing*), report the accuracy and four fairness metrics of the debiased predictions. Compare the results with the baseline generated in **Q1**, How do the accuracy and fairness metrics change？

**Q4 (10 points)**: Run the code in **step 5.3**, for the post-processing method (*Reject Option‐based Classiﬁcation*), report the accuracy and four fairness metrics of the debiased prediction. Compare the results with the baseline generated in **Q1**, How do the accuracy and fairness metrics change？

**Q5 (30 points)**: Following **step 6**, choose two additional combinations of debiasing techniques. For each combination, report the combination, the code, the accuracy and four fairness metrics of the final prediction, and compare the accuracy and four fairness metrics with the baseline generated in **Q1**.
*Reminder: For each combination, you should use at least two debiasing techniques.*

**Q6 (10 points)**: Compare all accuracies and fairness metrics you generated from **Q1**-**Q5**, what do you find about the relationship between fairness and accuracy?